In [1]:
# Import required libraries
import speech_recognition as sr
import pyttsx3
import pywhatkit
import webbrowser

In [2]:
# Initialize text-to-speech engine
def speak(text):
    """Convert text to speech."""
    # Reinitialize engine for each call to avoid runtime conflicts
    engine = pyttsx3.init()
    engine.setProperty("rate", 150)
    engine.setProperty("volume", 1.0)
    engine.say(text)
    engine.runAndWait()
    engine.stop()  # Stop the engine to prevent overlapping issues

def take_command():
    """Capture audio from the microphone and convert it to text."""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        speak("Listening...")
        recognizer.adjust_for_ambient_noise(source)
        try:
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=5)
            command = recognizer.recognize_google(audio)
            return command.lower()
        except sr.UnknownValueError:
            speak("Sorry, I didn't understand that.")
            return None
        except sr.RequestError:
            speak("Speech recognition service is not available.")
            return None
        except sr.WaitTimeoutError:
            speak("No input detected. Please try again.")
            return None

def process_command(command):
    """Process the voice command and execute actions."""
    if "play" in command:
        song = command.replace("play", "").strip()
        speak(f"Playing {song} on YouTube.")
        try:
            pywhatkit.playonyt(song)
            return True
        except Exception as e:
            speak(f"An error occurred: {e}")
            return False
    elif "open" in command:
        website = command.replace("open", "").strip()
        if not website.startswith("http"):
            website = "https://www." + website + ".com"
        speak(f"Opening {website}.")
        try:
            webbrowser.open(website)
            return True
        except Exception as e:
            speak(f"An error occurred: {e}")
            return False
    else:
        speak("Sorry, I can only play songs or open websites.")
        return False

In [3]:
# Main program logic
if __name__ == "__main__":
    speak("Welcome! I can play songs on YouTube or open websites for you.")
    
    # Ask for a command
    user_command = None
    while not user_command:
        speak("Please say a command.")
        user_command = take_command()

    # Process the command until fulfilled
    command_success = False
    while not command_success:
        command_success = process_command(user_command)
        if not command_success:
            speak("Please try again.")
            user_command = take_command()